Итоговое задание Виктора Андрийчука по Проекту 3. О вкусной и здоровой пище (SF-DST-14)

Юнит 3. Введение в машинное обучение (отредактирован 12.07.2020)

![](https://www.pata.org/wp-content/uploads/2014/09/TripAdvisor_Logo-300x119.png)
# Predict TripAdvisor Rating
## В этом соревновании нам предстоит предсказать рейтинг ресторана в TripAdvisor
**По ходу задачи:**
* Прокачаем работу с pandas
* Научимся работать с Kaggle Notebooks
* Поймем как делать предобработку различных данных
* Научимся работать с пропущенными данными (Nan)
* Познакомимся с различными видами кодирования признаков
* Немного попробуем [Feature Engineering](https://ru.wikipedia.org/wiki/Конструирование_признаков) (генерировать новые признаки)
* И совсем немного затронем ML
* И многое другое...   



### И самое важное, все это вы сможете сделать самостоятельно!

*Этот Ноутбук являетсся Примером/Шаблоном к этому соревнованию (Baseline) и не служит готовым решением!*   
Вы можете использовать его как основу для построения своего решения.

> что такое baseline решение, зачем оно нужно и почему предоставлять baseline к соревнованию стало важным стандартом на kaggle и других площадках.   
**baseline** создается больше как шаблон, где можно посмотреть как происходит обращение с входящими данными и что нужно получить на выходе. При этом МЛ начинка может быть достаточно простой, просто для примера. Это помогает быстрее приступить к самому МЛ, а не тратить ценное время на чисто инженерные задачи. 
Также baseline являеться хорошей опорной точкой по метрике. Если твое решение хуже baseline - ты явно делаешь что-то не то и стоит попробовать другой путь) 

В контексте нашего соревнования baseline идет с небольшими примерами того, что можно делать с данными, и с инструкцией, что делать дальше, чтобы улучшить результат.  Вообще готовым решением это сложно назвать, так как используются всего 2 самых простых признака (а остальные исключаются).

# import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import matplotlib.pyplot as plt
import seaborn as sns 
%matplotlib inline

import re
import datetime


# Загружаем специальный удобный инструмент для разделения датасета:
from sklearn.model_selection import train_test_split

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
# всегда фиксируйте RANDOM_SEED, чтобы ваши эксперименты были воспроизводимы!
RANDOM_SEED = 42
CURRENT_DATE = pd.to_datetime('12/07/2020')

In [ ]:
# зафиксируем версию пакетов, чтобы эксперименты были воспроизводимы:
!pip freeze > requirements.txt

# DATA

In [ ]:
DATA_DIR = '/kaggle/input/sf-dst-restaurant-rating/'
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'kaggle_task.csv')
sample_submission = pd.read_csv(DATA_DIR+'/sample_submission.csv')

In [ ]:
# ВАЖНО! дря корректной обработки признаков объединяем трейн и тест в один датасет
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем

In [ ]:
data

Подробнее по признакам:
* City: Город 
* Cuisine Style: Кухня
* Ranking: Ранг ресторана относительно других ресторанов в этом городе
* Price Range: Цены в ресторане в 3 категориях
* Number of Reviews: Количество отзывов
* Reviews: 2 последних отзыва и даты этих отзывов
* URL_TA: страница ресторана на 'www.tripadvisor.com' 
* ID_TA: ID ресторана в TripAdvisor
* Rating: Рейтинг ресторана

# Обработка

## 1. Обработка NAN 
У наличия пропусков могут быть разные причины, но пропуски нужно либо заполнить, либо исключить из набора полностью. Но с пропусками нужно быть внимательным, **даже отсутствие информации может быть важным признаком!**   
По этому перед обработкой NAN лучше вынести информацию о наличии пропуска как отдельный признак 

In [ ]:
# Добавим новый столбец для Number of Reviews и запишем туда пропуски, которые встречались в Number of Reviews
data['Number of Reviews_isNAN'] = pd.isna(data['Number of Reviews']).astype('uint8')

In [ ]:
# Далее заполняем пропуски 0. Скорее всего если отзывов нет, то вместо пропуска должен быть 0.
data['Number of Reviews'].fillna(0, inplace=True)

### 2. Обработка признаков
Для начала посмотрим какие признаки у нас могут быть категориальными.

In [ ]:
data.nunique(dropna=False)

Какие признаки можно считать категориальными?

Для кодирования категориальных признаков есть множество подходов:
* Label Encoding
* One-Hot Encoding
* Target Encoding
* Hashing

Выбор кодирования зависит от признака и выбраной модели.

### City

In [ ]:
data.sample()

In [ ]:
# Получается, что Ranking имеет нормальное распределение, 
# просто в больших городах больше ресторанов, из-за мы этого имеем смещение
# необходимо отнормировать критерий Ranking по городам City
mean_Ranking_on_City = data.groupby(['City'])['Ranking'].mean()
count_Restorant_in_City = data['City'].value_counts(ascending=False)
data['mean_Ranking_on_City'] = data['City'].apply(lambda x: mean_Ranking_on_City[x])
data['count_Restorant_in_City'] = data['City'].apply(lambda x: count_Restorant_in_City[x])
data['norm_Ranking_on_Rest_in_City'] = (data['Ranking'] - data['mean_Ranking_on_City']) / data['count_Restorant_in_City']

In [ ]:
max_Ranking_on_City = data.groupby(['City'])['Ranking'].max()
data['max_Ranking_on_City'] = data['City'].apply(lambda x: max_Ranking_on_City[x])
data['norm_Ranking_on_maxRank_in_City'] = (data['Ranking'] - data['mean_Ranking_on_City']) / data['max_Ranking_on_City']

In [ ]:
# для One-Hot Encoding в pandas есть готовая функция - get_dummies. Особенно радует параметр dummy_na
data = pd.get_dummies(data, columns=[ 'City',], dummy_na=True)

### Price Range

In [ ]:
data['Price Range'].value_counts()

По описанию 'Price Range' это - Цены в ресторане.  
Их можно поставить по возрастанию (значит это не категориальный признак). А это значит, что их можно заменить последовательными числами, например 1,2,3

In [ ]:
# Обработка 'Price Range' - замена по словарю

pr_dict = {'$': 1, '$$ - $$$': 2, '$$$$': 3}

data = data.replace({"Price Range": pr_dict})

In [ ]:
# А пропуски в 'Price Range' заполним нулями. Предварительно создам создам отдельную колонку и запишу где были пропуски.
data['Price Range_isNAN'] = pd.isna(data['Price Range']).astype('uint8')
data['Price Range'].fillna(0, inplace=True)

In [ ]:
data['Price Range'].value_counts()

In [ ]:
data.sample(5)

### General

In [ ]:
# Очищаем данные от мусора. Для начала список строк, которые мне вроде как вообще пока не нужны
data = data.drop(['URL_TA', 'ID_TA'], axis = 1)

In [ ]:
# Приведу типы колонок к int, где точно int 
data = data.astype({"Price Range": int,"Number of Reviews": int})

### Cuisine Style

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Для начала создам отдельную колонку и запишу туда все пропуски, которые встречались в Cuisine Style
data['Cuisine Style_isNAN'] = pd.isna(data['Cuisine Style']).astype('float64') 

# Заполню пропуски значением 'Other' - пусть такие кухни будут в одной категории. 
data['Cuisine Style'] = data['Cuisine Style'].fillna("['Other']")

In [ ]:
# Формирую новые колонки исходя из кухонь в  Cuisine Style

from yaml import safe_load 
from sklearn.preprocessing import MultiLabelBinarizer

mlb = MultiLabelBinarizer(sparse_output=True)

tmp = data["Cuisine Style"].dropna().apply(safe_load).dropna()

X = pd.DataFrame.sparse.from_spmatrix(
        mlb.fit_transform(tmp), 
        columns=mlb.classes_, 
        index=tmp.index)

data_new = data.join(X, how="left")

In [ ]:
data_new.sample(5)

In [ ]:
# data_new.iloc[:,46:]
# Создам отдельную колонку-признак - кол-во кухонь в ресторане
data_new["Count of Cuisines"] = data_new.iloc[:,46:].sum(axis=1)

In [ ]:
# Думаю, можно уже удалить колонку Cuisine Style
data_new = data_new.drop(['Cuisine Style'], axis = 1)

In [ ]:
data_new

### Restaurant_id

In [ ]:
# Пока мысли - странно во многих строках данные в Restaurant_id и Ranking визуально как бы коррелируют.
# Запомним этот момент. Позже гляну на корреляция и если что, то удалю Restaurant_id

data_new['Restaurant_id_corr_with_Ranking'] = data_new['Restaurant_id'].apply(lambda x: float(x[3:]))

In [ ]:
data_new.sample(5)

In [ ]:
data_new.info()

In [ ]:
# С помощью этого я приведу дозаполню пустые места в кухнях и приведу колонки к типу "число"

data_new.iloc[:,42:-2] = np.nan_to_num(data_new.iloc[:,42:-2], nan=0)
data_new.iloc[:,42:-2] = data_new.iloc[:,42:-2].fillna(0)
data_new.iloc[:,42:-2] = data_new.iloc[:,42:-2].astype(int)

In [ ]:
data_new.info()

In [ ]:
# Удаляю Restaurant_id и Restaurant_id_corr_with_Ranking
data_new = data_new.drop(['Restaurant_id', 'Restaurant_id_corr_with_Ranking'], axis = 1)

In [ ]:
data_new

### Reviews

In [ ]:
# В Reviews нет пропусков, но 6471 строк со значением [[], []]. По сути это пустые строки сохраним их 
data_new['empty_Reviews'] = (data_new['Reviews']=='[[], []]').astype('float64')

# анализ тестовой базы выявил два пропуска, несмотря на то, что pandas.profiling на тренировочной базе пропусков не выявил, заполним их '[[], []]' и закинем в empty_Reviews
data_new['Reviews'] = data_new['Reviews'].fillna('[[], []]')
data_new['empty_Reviews'] = (data_new['Reviews']=='[[], []]').astype('float64')

In [ ]:
data_new['date_of_Review'] = data_new['Reviews'].str.findall('\d+/\d+/\d+')
data_new['len_date'] = data_new['date_of_Review'].apply(lambda x: len(x))



In [ ]:
# есть значение 3 надо разобраться что там
print("кол-во значений Reviews с тремя датами :=" , len(data_new[data_new['len_date']==3]))
print("значения Reviews с тремя датами :=")
temp_list = data_new[data_new['len_date']==3].Reviews.to_list()
display(data_new[data_new['len_date']==3].Reviews.to_list())
print("даты после обработки регулярными выражениями:")
display([re.findall('\d+/\d+/\d+', x) for x in temp_list])

In [ ]:
# видим что люди указывали даты в отзывах и эти даты попали в обработку
# из-за этого возникнут ошибки так как даты не верные и их формат отличается и формата выгрузки
# при этом таких строк всего четыре (4), можно было бы их не исправлять а выбросить потому что 17 
# год явно приведет к выбросу с которым надо будет разбираться. Выбрасывать жалко, тогда исправим,
# тем более, что это достачно просто

data_new['len_date'].date_of_Review = data_new[data_new['len_date']==3].date_of_Review.apply(lambda x: x.pop(0))

In [ ]:
# также есть значение 1 надо разобраться что там
print("кол-во значений Reviews с одной датой :=" , len(data_new[data_new['len_date']==1]))
display(data_new[data_new['len_date']==1].Reviews[:4])

In [ ]:
# оказалось, что есть отзывы с одним (1) отзывом и их достаточно много 5680 из (40000-6471) это 17%
# сохраним это на всякий случай, чтобы не потерять
data_new['one_Review'] = (data_new['len_date']==1).astype('float64')

# заполним перерыв между отзывами (по отзывам где len = 2) и насколько давно был сделан последний самый свежий отзыв
# создадим для этого функции:
def time_to_now(row):
    if row['date_of_Review'] == []:
        return None
    return datetime.datetime.now() - pd.to_datetime(row['date_of_Review']).max()

def time_between_Reviews(row):
    if row['date_of_Review'] == []:
        return None
    return pd.to_datetime(row['date_of_Review']).max() - pd.to_datetime(row['date_of_Review']).min()

data_new['day_to_now'] = data_new.apply(time_to_now, axis = 1).dt.days
data_new['day_between_Reviews'] = data_new[data_new['len_date']==2].apply(time_between_Reviews, axis = 1).dt.days

Резюме - Reviews: Пропусков в тренировочном датасете нет, в тестовом - 2. Но есть 6471 незаполненных строк с отзывами в тренировочном датасете это 16% от датасета. В 5680 (14%) строках есть только один отзыв, хотя в подавляющем большинстве отзывов два.
Созданы новые критерии:

empty_Reviews - незаполненные отзывы
date_of_Review - даты из отзывов
len_date - кол-во дат в отзыве
day_to_now - насколько давно был сделан последний самый свежий отзыв в днях
day_between_Reviews - перерыв между отзывами в днях
Резюме - day_to_now из Reviews: Удаление по порогу не напрашивается так как компания TripAdvisor работает с 2000 года. Максимум 5896/365 ~ 16,5 лет от 2020 года укладывается в дату начала старта сайта. В выбросы попало 2365 (почти 6%) значений, с учетом резюме по неполным данным в критерии Reviews, я пока принимаю решение не избавлятся от выросов, построить модель, обратить внимание на важность критерия, и при необходимости вернуться к нему для заполнения парсингом или удаления выбросов

In [ ]:
# кол-во выбросов 495 (1.2%) - это статистически не значимо, но мы пока сохраняем информацию о выбросе, а потом проверим его важность в модели
data_new['out_day_between_Reviews'] = (data_new['day_between_Reviews']==0).astype('float64')

# и удаляем выбросы
data_new.loc[data_new['day_between_Reviews']==0, 'day_between_Reviews'] = None

Резюме - day_between_Reviews из Reviews: Заполнен слабо 70%. Удалены выбросы в нуле (492 значения). Создан новый критерий - out_day_between_Reviews

In [ ]:
data_new.sample(5)

### Number of Reviews

In [ ]:
# в переменной 2543 (6.4%) пропущенных значений 
# сохраним эту информацию
data_new['Number of Reviews_NAN'] = pd.isna(data_new['Number of Reviews']).astype('float64')

In [ ]:
# выбросов не так много, удалим их, предварительно сохранив информацию о них
data_new['outliers_Number of Reviews'] = pd.DataFrame(data_new['Number of Reviews']>5252).astype('float64')
data_new.loc[data_new['Number of Reviews']>5252, 'Number of Reviews']=None

Резюме - Number of Reviews 2543 (6.4%) пропусков.

In [ ]:
data_new.select_dtypes(include=['object'])

In [ ]:
# Удалим оставшиеся object строки
data_new = data_new.drop(['Reviews', 'date_of_Review'], axis = 1)

In [ ]:
data_new.info()

In [ ]:
data_new.isnull().sum().sum()

In [ ]:
data_new.fillna(0, inplace=True)

In [ ]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

def applyFeatures(dataset):

    columns = dataset.columns.tolist()
    target = columns[4]
    columns.remove(target)
    
    
    for c in columns:
        dataset[c] = scaler.fit_transform(dataset[c].values.reshape(-1, 1))
    
        
applyFeatures(data_new)

In [ ]:
 #MinMax Normalization not running for this algorithm

    from sklearn.preprocessing import MinMaxScaler

    scaler = MinMaxScaler()
    data_new[['Ranking']] = scaler.fit_transform(data_new[['Ranking']])

   

In [ ]:
data_new

# EDA

### Посмотрим на корреляция между Restaurant_id и ranking

In [ ]:
data_new.corr()

Корреляция между Ranking и Restaurant_id_corr_with_Ranking достаточно большая. Принимаю решение удалить Restaurant_id_corr_with_Ranking и следовательно Restaurant_id

### Посмотрим распределение признака

In [ ]:
plt.rcParams['figure.figsize'] = (10,7)
df_train['Ranking'].hist(bins=100)

У нас много ресторанов, которые не дотягивают и до 2500 места в своем городе, а что там по городам?

In [ ]:
df_train['City'].value_counts(ascending=True).plot(kind='barh')

А кто-то говорил, что французы любят поесть=) Посмотрим, как изменится распределение в большом городе:

In [ ]:
df_train['Ranking'][df_train['City'] =='London'].hist(bins=100)

In [ ]:
# посмотрим на топ 10 городов
for x in (df_train['City'].value_counts())[0:10].index:
    df_train['Ranking'][df_train['City'] == x].hist(bins=100)
plt.show()

Получается, что Ranking имеет нормальное распределение, просто в больших городах больше ресторанов, из-за мы этого имеем смещение.

### Посмотрим распределение целевой переменной

In [ ]:
df_train['Rating'].value_counts(ascending=True).plot(kind='barh')

### Посмотрим распределение целевой переменной относительно признака

In [ ]:
df_train['Ranking'][df_train['Rating'] == 5].hist(bins=100)

In [ ]:
df_train['Ranking'][df_train['Rating'] < 4].hist(bins=100)

### И один из моих любимых - [корреляция признаков](https://ru.wikipedia.org/wiki/Корреляция)
На этом графике уже сейчас вы сможете заметить, как признаки связаны между собой и с целевой переменной.

In [ ]:
plt.rcParams['figure.figsize'] = (15,10)
sns.heatmap(data_new.drop(['sample'], axis=1).corr(),)

Вообще благодаря визуализации в этом датасете можно узнать много интересных фактов, например:
* где больше Пицерий в Мадриде или Лондоне?
* в каком городе кухня ресторанов более разнообразна?

придумайте свои вопрос и найдите на него ответ в данных)

# Data Preprocessing
Теперь, для удобства и воспроизводимости кода, завернем всю обработку в одну большую функцию.

In [ ]:
# на всякий случай, заново подгружаем данные
df_train = pd.read_csv(DATA_DIR+'/main_task.csv')
df_test = pd.read_csv(DATA_DIR+'/kaggle_task.csv')
df_train['sample'] = 1 # помечаем где у нас трейн
df_test['sample'] = 0 # помечаем где у нас тест
df_test['Rating'] = 0 # в тесте у нас нет значения Rating, мы его должны предсказать, по этому пока просто заполняем нулями

data = df_test.append(df_train, sort=False).reset_index(drop=True) # объединяем
data.info()

In [ ]:
def preproc_data(df_input):
    '''includes several functions to pre-process the predictor data.'''
    
    df_output = df_input.copy()
    
    # ################### 1. Предобработка ############################################################## 
    
    
    # Заполню это позже когда отработаю до конца все.
   
    
    # ################### 7. Clean #################################################### 
    # убираем признаки которые еще не успели обработать, 
    # модель на признаках с dtypes "object" обучаться не будет, просто выберим их и удалим
    object_columns = [s for s in df_output.columns if df_output[s].dtypes == 'object']
    df_output.drop(object_columns, axis = 1, inplace=True)
    
    return df_output

>По хорошему, можно было бы перевести эту большую функцию в класс и разбить на подфункции (согласно ООП). 

#### Запускаем и проверяем что получилось

In [ ]:
df_preproc = preproc_data(data)
df_preproc.sample(10)

In [ ]:
df_preproc.info()

In [ ]:
df_preproc

In [ ]:
# Теперь выделим тестовую часть
train_data = data_new.query('sample == 1.0').drop(['sample'], axis=1)
test_data = data_new.query('sample == 0.0').drop(['sample'], axis=1)

y = train_data.Rating.values            # наш таргет
X = train_data.drop(['Rating'], axis=1)

**Перед тем как отправлять наши данные на обучение, разделим данные на еще один тест и трейн, для валидации. 
Это поможет нам проверить, как хорошо наша модель работает, до отправки submissiona на kaggle.**

In [ ]:
# Воспользуемся специальной функцие train_test_split для разбивки тестовых данных
# выделим 20% данных на валидацию (параметр test_size)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [ ]:
# проверяем
test_data.shape, train_data.shape, X.shape, X_train.shape, X_test.shape

# Model 
Сам ML

In [ ]:
# Импортируем необходимые библиотеки:
from sklearn.ensemble import RandomForestRegressor # инструмент для создания и обучения модели
from sklearn import metrics # инструменты для оценки точности модели

In [ ]:
# Создаём модель (НАСТРОЙКИ НЕ ТРОГАЕМ)
model = RandomForestRegressor(n_estimators=100, verbose=1, n_jobs=-1, random_state=RANDOM_SEED)

In [ ]:
# Обучаем модель на тестовом наборе данных
model.fit(X_train, y_train)

# Используем обученную модель для предсказания рейтинга ресторанов в тестовой выборке.
# Предсказанные значения записываем в переменную y_pred
y_pred = model.predict(X_test)

In [ ]:
# Сравниваем предсказанные значения (y_pred) с реальными (y_test), и смотрим насколько они в среднем отличаются
# Метрика называется Mean Absolute Error (MAE) и показывает среднее отклонение предсказанных значений от фактических.
print('MAE:', metrics.mean_absolute_error(y_test, y_pred))

In [ ]:
# в RandomForestRegressor есть возможность вывести самые важные признаки для модели
plt.rcParams['figure.figsize'] = (10,10)
feat_importances = pd.Series(model.feature_importances_, index=X.columns)
feat_importances.nlargest(15).plot(kind='barh')

# Submission
Если все устраевает - готовим Submission на кагл

In [ ]:
test_data.sample(10)

In [ ]:
test_data = test_data.drop(['Rating'], axis=1)

In [ ]:
sample_submission

In [ ]:
predict_submission = model.predict(test_data)

In [ ]:
predict_submission

In [ ]:
sample_submission['Rating'] = predict_submission
sample_submission.to_csv('submission.csv', index=False)
sample_submission.head(10)

# What's next?
Или что делать, чтоб улучшить результат:
* Обработать оставшиеся признаки в понятный для машины формат
* Посмотреть, что еще можно извлечь из признаков
* Сгенерировать новые признаки
* Подгрузить дополнительные данные, например: по населению или благосостоянию городов
* Подобрать состав признаков

В общем, процесс творческий и весьма увлекательный! Удачи в соревновании!
